# 🚗 CAR PURCHASE DEMAND SIMULATOR
**PHẦN 3: TRIỂN KHAI ỨNG DỤNG WEB (WEB APP DEPLOYMENT)**  
*(Thuộc dự án: CAR PURCHASE DECISION ANALYSIS - XAI)*

---

### 👤 AUTHOR INFO
* **Tác giả:** Bùi Tiến Phát (Higo)
* **Email:** higo.individual@gmail.com
* **Project Type:** Model Deployment / Web Application
* **Kỹ thuật:** Streamlit, Pyngrok (Secure Tunneling), Automated Artifact Retrieval.

---

### 📝 MODULE HIGHLIGHTS (TÍNH NĂNG NỔI BẬT):

* **Automated Artifact Retrieval (Tự động hóa tài nguyên):** Thiết lập cơ chế tự động tải mô hình (`.joblib`), cấu trúc cột và dữ liệu từ **GitHub Raw URL**, loại bỏ hoàn toàn thao tác upload thủ công và đảm bảo tính đồng bộ của hệ thống.
* **Real-time Inference Engine (Cơ chế suy luận thời gian thực):** Xây dựng giao diện tương tác chuyển đổi thông số đầu vào (Giá, Mã lực, Hộp số...) thành vector đặc trưng chuẩn hóa (One-Hot Encoding) để mô hình Random Forest thực hiện dự báo tức thì.
* **Dynamic Data Sanitization (Làm sạch dữ liệu động):** Tích hợp bộ lọc logic ngay trên ứng dụng để xử lý các giá trị nhiễu còn sót lại (ví dụ: loại bỏ `Automatic` khỏi danh mục Nhiên liệu), đảm bảo tính hợp lệ của dữ liệu đầu vào mà không cần can thiệp vào file gốc.
* **Secure Public Tunneling (Triển khai đường dẫn bảo mật):** Sử dụng `pyngrok` kết hợp với cơ chế nhập Token ẩn (`getpass`) để public server cục bộ (Localhost) ra Internet một cách an toàn, phục vụ mục đích Demo và báo cáo.

**BƯỚC 1: CÀI ĐẶT MÔI TRƯỜNG**  
Mục đích: Cài đặt các thư viện cần thiết để chạy Web App và tạo đường hầm kết nối ra Internet.

In [1]:
# Cài đặt thư viện
!pip install -q streamlit pyngrok joblib pandas numpy scikit-learn
print("✅ Môi trường đã sẵn sàng!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.1 MB/s eta 0:00:00
✅ Đã cài đặt môi trường thành công!


**BƯỚC 2: ĐỒNG BỘ DỮ LIỆU TỰ ĐỘNG**  
Mục đích: Tự động tải file Model (.joblib) và Dữ liệu (.csv) từ GitHub.

In [2]:
import os

# 1. Dọn dẹp file cũ (nếu có)
files_to_remove = ["app.py", "car_purchase_model.joblib", "model_columns.joblib", "processed_carbuyers.csv"]
for f in files_to_remove:
    if os.path.exists(f):
        os.remove(f)

# 2. Định nghĩa Link GitHub (Raw)
repo_url = "https://raw.githubusercontent.com/higo-ai/Car-Purchase-Decision-Analysis-XAI/main"
downloads = {
    "car_purchase_model.joblib": f"{repo_url}/models/car_purchase_model.joblib",
    "model_columns.joblib": f"{repo_url}/models/model_columns.joblib",
    "processed_carbuyers.csv": f"{repo_url}/data/processed_carbuyers.csv"
}

# 3. Tải file
print("--> ⏳ Đang tải tài nguyên từ GitHub...")
for filename, url in downloads.items():
    exit_code = os.system(f"wget {url} -O {filename}")
    if exit_code == 0:
        print(f"   ✅ Đã tải thành công: {filename}")
    else:
        print(f"   ❌ Lỗi tải file: {filename} (Kiểm tra lại link GitHub)")

print("\n🚀 Dữ liệu đã sẵn sàng! Chuyển sang bước tiếp theo.")

--> ⏳ Đang tải tài nguyên từ GitHub...
   ✅ Đã tải thành công: car_purchase_model.joblib
   ✅ Đã tải thành công: model_columns.joblib
   ✅ Đã tải thành công: processed_carbuyers.csv

🚀 Dữ liệu đã sẵn sàng! Chuyển sang bước tiếp theo.


**BƯỚC 3: XÂY DỰNG GIAO DIỆN APP**  
Mục đích: Tạo file app.py chứa toàn bộ logic.

In [3]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import joblib

# 1. CẤU HÌNH GIAO DIỆN
st.set_page_config(page_title="Phân Tích Quyết Định Mua Xe", page_icon="🚗", layout="wide")

st.markdown("""
<style>
    .main-header {font-size: 2.2rem; color: #0068C9; font-weight: bold; text-align: center; margin-bottom: 20px;}
    .sub-text {font-size: 1.1rem; color: #555; text-align: center; font-style: italic;}
    .metric-box {background-color: #F0F2F6; padding: 20px; border-radius: 10px; border-left: 5px solid #0068C9;}
    .stButton>button {background-color: #0068C9; color: white; width: 100%; font-weight: bold; height: 50px; border-radius: 8px;}
</style>
""", unsafe_allow_html=True)

st.markdown('<div class="main-header">🚗 HỆ THỐNG MÔ PHỎNG SỨC MUA Ô TÔ</div>', unsafe_allow_html=True)
st.markdown('<div class="sub-text">Phân tích độ nhạy: Thay đổi các yếu tố kỹ thuật để xem tác động đến quyết định mua (Total)</div>', unsafe_allow_html=True)
st.markdown("---")

# 2. HÀM TẢI TÀI NGUYÊN (CACHED)
@st.cache_resource
def load_resources():
    try:
        df = pd.read_csv('processed_carbuyers.csv')
        model = joblib.load('car_purchase_model.joblib')

        # Load danh sách cột (Ưu tiên file columns riêng)
        try:
            model_cols = joblib.load('model_columns.joblib')
        except:
            model_cols = model.feature_names_in_ if hasattr(model, 'feature_names_in_') else None

        return df, model, model_cols
    except Exception as e:
        return None, None, None

df_org, model, model_cols = load_resources()

# 3. THANH ĐIỀU KHIỂN (SIDEBAR)
if df_org is not None and model is not None:
    st.sidebar.header("🛠️ THIẾT LẬP THÔNG SỐ XE")

    # --- A. CHỌN HÃNG XE ---
    manus = sorted(df_org['Manufacturer'].unique().tolist())
    if 'Ford' in manus: manus.insert(0, manus.pop(manus.index('Ford')))
    sel_manu = st.sidebar.selectbox("Hãng xe (Manufacturer)", manus)

    # --- B. CHỌN NHIÊN LIỆU ---
    # Chỉ lấy các loại nhiên liệu thực tế, loại bỏ dòng lỗi 'Automatic' để UI sạch đẹp
    raw_fuels = [x for x in df_org['Fuel'].unique() if isinstance(x, str) and x.lower() != 'automatic']
    sel_fuel = st.sidebar.selectbox("Nhiên liệu (Fuel Type)", raw_fuels)

    st.sidebar.markdown("---")

    # --- C. THÔNG SỐ KỸ THUẬT ---
    min_price, max_price = int(df_org['Price'].min()), int(df_org['Price'].max())
    sel_price = st.sidebar.slider(f"Giá bán (Price - Nghìn USD)", min_price, max_price, 25)

    min_power, max_power = int(df_org['Power'].min()), int(df_org['Power'].max())
    sel_power = st.sidebar.slider("Công suất (Power - Mã lực/HP)", min_power, max_power, 150)

    min_engine, max_engine = int(df_org['Engine CC'].min()), int(df_org['Engine CC'].max())
    sel_engine = st.sidebar.number_input("Dung tích (Engine CC)", min_engine, max_engine, 2000)

    # Hộp số (Slider 0-10)
    sel_trans = st.sidebar.slider("Hộp số (Transmission - Số cấp)", 0, 10, 5)

    # 4. XỬ LÝ DỰ BÁO
    if st.sidebar.button("🚀 CHẠY MÔ PHỎNG PHÂN TÍCH"):
        # Tạo khung dữ liệu chuẩn theo Model
        input_df = pd.DataFrame(0, index=[0], columns=model_cols)

        # Điền các biến số (Numeric)
        if 'Price' in model_cols: input_df['Price'] = sel_price
        if 'Power' in model_cols: input_df['Power'] = sel_power
        if 'Engine CC' in model_cols: input_df['Engine CC'] = sel_engine

        # Xử lý Hộp số
        if 'Transmission' in model_cols:
            input_df['Transmission'] = sel_trans

        # Điền biến phân loại (One-Hot Encoding)
        # Tự động tìm cột đúng (VD: Manufacturer_Ford) để điền số 1
        if f"Manufacturer_{sel_manu}" in model_cols: input_df[f"Manufacturer_{sel_manu}"] = 1
        if f"Fuel_{sel_fuel}" in model_cols: input_df[f"Fuel_{sel_fuel}"] = 1

        try:
            # Thực hiện dự báo
            prediction = model.predict(input_df)[0]

            # HIỂN THỊ KẾT QUẢ
            c1, c2 = st.columns([1.5, 1])
            with c1:
                st.markdown("### 📊 KẾT QUẢ DỰ BÁO")
                st.markdown(f"""
                <div class="metric-box">
                    <h4 style="margin:0; color:#555">SỨC MUA DỰ KIẾN (TOTAL)</h4>
                    <h1 style="margin:0; color:#0068C9">{prediction:,.2f}</h1>
                    <p>Khách hàng tiềm năng</p>
                </div>
                """, unsafe_allow_html=True)

                avg = df_org['Total'].mean()
                diff = prediction - avg

                st.write("")
                if diff > 0:
                    st.success(f"📈 **Tiềm năng:** Cấu hình này thu hút cao hơn trung bình thị trường (+{diff:,.0f}).")
                else:
                    st.warning(f"📉 **Rủi ro:** Cấu hình này có sức hút thấp hơn trung bình thị trường ({diff:,.0f}).")

            with c2:
                st.markdown("### 🔍 SO SÁNH")
                chart_data = pd.DataFrame({"Loại": ["Trung bình TT", "Cấu hình này"], "Total": [avg, prediction]})
                st.bar_chart(chart_data.set_index("Loại"), color="#0068C9")

        except Exception as e:
            st.error(f"Lỗi xử lý dự báo: {str(e)}")
else:
    st.info("⏳ Đang tải dữ liệu... Vui lòng đợi trong giây lát.")

Writing app.py


**BƯỚC 4: KÍCH HOẠT SERVER**  
Mục đích: Chạy ứng dụng Streamlit ngầm (background) và mở cổng kết nối an toàn qua Ngrok truy cập từ trình duyệt.

In [5]:
# 4 Chạy Ngrok (Phiên bản Bảo Mật)
from pyngrok import ngrok
import os
import time
import getpass

# 1. Nhập Token an toàn (Không hiện lên màn hình)
print("🔐 Hãy dán Ngrok Authtoken vào ô bên dưới và bấm Enter:")
ngrok_token = getpass.getpass()

# 2. Setup Token
ngrok.set_auth_token(ngrok_token)

# 3. Kill process cũ
!pkill streamlit
ngrok.kill()

# 4. Chạy Streamlit background
print("--> 🚀 Đang khởi động Server...")
os.system("streamlit run app.py --server.port 8501 &")

# 5. Đợi 3s và mở kết nối
time.sleep(3)

try:
    public_url = ngrok.connect(8501).public_url
    print("🎉 HOÀN TẤT! APP ĐANG CHẠY TẠI:")
    print(f"👉 {public_url}")
    print("💡 Click vào link trên để mở Web App.")
except Exception as e:
    print(f"❌ Lỗi kết nối Ngrok: {e}")

🔐 Hãy dán Ngrok Authtoken vào ô bên dưới và bấm Enter:
··········
--> 🚀 Đang khởi động Server...
🎉 HOÀN TẤT! APP ĐANG CHẠY TẠI:
👉 https://unresented-josette-untemporary.ngrok-free.dev
💡 Click vào link trên để mở Web App.
